# Validação de Resultados de Detecção

Este notebook valida as detecções do sistema comparando-as com o ground truth manual.

**Objetivo:** Alcançar acima de 90% de precisão, poucos falsos positivos e acertar tudo.

**Métricas calculadas:**
- Precisão (Precision)
- Recall (Cobertura)
- F1-Score
- True Positives (TP)
- False Positives (FP)
- False Negatives (FN)

**Melhorias no Matching (v2):**
- Ignora método quando é 'file_level' (genérico usado pelo sistema)
- Tolerância de ±2 linhas para Line_no
- Compara apenas nome do arquivo (não caminho completo)
- Cada item do GT só pode ser matched uma vez

**Ground Truth Revisado:**
- Removidas 91 duplicatas
- Removidas 213 entradas falsas de Long Message Chain (chamadas de módulo, não cadeias de métodos)


In [121]:
from pathlib import Path

import pandas as pd

# Configurações
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

BASE_DIR = Path().resolve().parent
RESULTS_DIR = BASE_DIR / "results"
DATASET_DIR = BASE_DIR / "dataset"

GROUND_TRUTH_CSV = DATASET_DIR / "ground_truth" / "ground_truth_corrected.csv"
RESULTS_SIMPLE_CSV = RESULTS_DIR / "csv" / "results_simple_prompt.csv"
RESULTS_COMPLETE_CSV = RESULTS_DIR / "csv" / "results_with_complete_prompts.csv"
RESULTS_SIMPLE_JSON = RESULTS_DIR / "json" / "results_simple_prompt.json"
RESULTS_COMPLETE_JSON = RESULTS_DIR / "json" / "results_with_complete_prompts.json"

print(f"Base dir: {BASE_DIR}")
print(f"Ground truth CSV: {GROUND_TRUTH_CSV.exists()}")
print(f"Results simple CSV: {RESULTS_SIMPLE_CSV.exists()}")
print(f"Results complete CSV: {RESULTS_COMPLETE_CSV.exists()}")


Base dir: /home/luis-chaves/Área de trabalho/tcc/multi-agent-smell-detector
Ground truth CSV: True
Results simple CSV: True
Results complete CSV: True


## 2. Processar DF's


In [122]:
df_gt = pd.read_csv(GROUND_TRUTH_CSV)
df_gt.head()

,Source,File,Method,Code_Smell,Line_no,Start_line,End_line,Details
0,dataset,_codespell.py,parse_file,Complex Conditional,991.0,NaN,NaN,Conditional at line 991 has 3 logical operator...
1,dataset,_codespell.py,open_with_internal,Complex Method,NaN,263.0,288.0,Method 'open_with_internal' has cyclomatic com...
2,dataset,_codespell.py,parse_options,Complex Method,NaN,375.0,697.0,Method 'parse_options' has cyclomatic complexi...
3,dataset,_codespell.py,ask_for_word_fix,Complex Method,NaN,754.0,813.0,Method 'ask_for_word_fix' has cyclomatic compl...
4,dataset,_codespell.py,parse_file,Complex Method,NaN,873.0,1084.0,Method 'parse_file' has cyclomatic complexity ...


In [123]:
df_gt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Source      397 non-null    object 
 1   File        397 non-null    object 
 2   Method      275 non-null    object 
 3   Code_Smell  397 non-null    object 
 4   Line_no     280 non-null    float64
 5   Start_line  93 non-null     float64
 6   End_line    93 non-null     float64
 7   Details     397 non-null    object 
dtypes: float64(3), object(5)
memory usage: 24.9+ KB


In [124]:
df_complete_prompts = pd.read_csv(RESULTS_COMPLETE_CSV)
df_complete_prompts.head()

,Source,File,Method,Code_Smell,Line_no,Start_line,End_line,Details
0,dataset,ff_ippo_store_experience.py,_update_step,Complex method,NaN,68.0,308.0,Method '_update_step' has cyclomatic complexit...
1,dataset,ff_ippo_store_experience.py,learner_setup,Complex method,NaN,346.0,448.0,Method 'learner_setup' has cyclomatic complexi...
2,dataset,ff_ippo_store_experience.py,run_experiment,Complex method,NaN,451.0,670.0,Method 'run_experiment' has cyclomatic complex...
3,dataset,ff_ippo_store_experience.py,_update_step,Long method,NaN,68.0,308.0,Method '_update_step' has 241 lines (threshold...
4,dataset,ff_ippo_store_experience.py,learner_setup,Long method,NaN,346.0,448.0,Method 'learner_setup' has 103 lines (threshol...


In [125]:
df_complete_prompts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Source      432 non-null    object 
 1   File        432 non-null    object 
 2   Method      432 non-null    object 
 3   Code_Smell  432 non-null    object 
 4   Line_no     275 non-null    float64
 5   Start_line  84 non-null     float64
 6   End_line    84 non-null     float64
 7   Details     432 non-null    object 
dtypes: float64(3), object(5)
memory usage: 27.1+ KB


In [126]:
df_simple_prompts = pd.read_csv(RESULTS_SIMPLE_CSV)
df_simple_prompts.head()

,Source,File,Method,Code_Smell,Line_no,Start_line,End_line,Details
0,dataset,ff_ippo_store_experience.py,_update_step,Complex method,NaN,68.0,308.0,Method '_update_step' has cyclomatic complexit...
1,dataset,ff_ippo_store_experience.py,_update_epoch,Complex method,NaN,154.0,296.0,Method '_update_epoch' has cyclomatic complexi...
2,dataset,ff_ippo_store_experience.py,get_learner_fn,Long method,NaN,57.0,343.0,Method 'get_learner_fn' has 281 lines (thresho...
3,dataset,ff_ippo_store_experience.py,learner_setup,Long method,NaN,346.0,448.0,Method 'learner_setup' has 102 lines (threshol...
4,dataset,ff_ippo_store_experience.py,run_experiment,Long method,NaN,451.0,671.0,Method 'run_experiment' has 220 lines (thresho...


In [127]:
df_simple_prompts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Source      919 non-null    object 
 1   File        919 non-null    object 
 2   Method      919 non-null    object 
 3   Code_Smell  919 non-null    object 
 4   Line_no     870 non-null    float64
 5   Start_line  48 non-null     float64
 6   End_line    48 non-null     float64
 7   Details     919 non-null    object 
dtypes: float64(3), object(5)
memory usage: 57.6+ KB


## 3. Converter colunas de linhas para inteiro


In [128]:
# Função para limpar valores de linha (remover caracteres como '|')
def clean_line_value(val):
    """Remove caracteres não numéricos de valores de linha."""
    if pd.isna(val):
        return val
    # Se for string, extrair apenas números
    if isinstance(val, str):
        import re
        match = re.search(r'\d+', val)
        return float(match.group()) if match else None
    return val

# Converter colunas de linhas de float64 para Int64 (nullable) nos três DataFrames
colunas_linhas = ["Line_no", "Start_line", "End_line"]

for col in colunas_linhas:
    df_complete_prompts[col] = df_complete_prompts[col].apply(clean_line_value)
    df_simple_prompts[col] = df_simple_prompts[col].apply(clean_line_value)

for col in colunas_linhas:
    df_gt[col] = df_gt[col].astype("Int64")

for col in colunas_linhas:
    df_complete_prompts[col] = df_complete_prompts[col].astype("Int64")

for col in colunas_linhas:
    df_simple_prompts[col] = df_simple_prompts[col].astype("Int64")

print("Tipos após conversão:")
print("\nGround Truth:")
print(df_gt[colunas_linhas].dtypes)
print("\nComplete Prompts:")
print(df_complete_prompts[colunas_linhas].dtypes)
print("\nSimple Prompts:")
print(df_simple_prompts[colunas_linhas].dtypes)


Tipos após conversão:

Ground Truth:
Line_no       Int64
Start_line    Int64
End_line      Int64
dtype: object

Complete Prompts:
Line_no       Int64
Start_line    Int64
End_line      Int64
dtype: object

Simple Prompts:
Line_no       Int64
Start_line    Int64
End_line      Int64
dtype: object


## 4. Funções de Validação


In [129]:
def normalize_smell_name(smell_name):
    """Normaliza o nome do smell para comparação (lowercase, remove espaços extras)."""
    if pd.isna(smell_name):
        return None
    return str(smell_name).lower().strip()


# Tolerância de linhas para matching (±N linhas)
LINE_TOLERANCE = 2


def match_detection(detection, gt_row):
    """
    Verifica se uma detecção faz match com um item do ground truth.
    
    Melhorias aplicadas:
    - Ignora método quando um deles é 'file_level' (genérico)
    - Tolerância de ±2 linhas para Line_no
    - Compara apenas nome do arquivo (não caminho completo)

    Args:
        detection: linha do DataFrame de detecções
        gt_row: linha do DataFrame de ground truth

    Returns:
        True se houver match, False caso contrário
    """
    # 1. Comparar arquivo (apenas nome, não caminho completo)
    det_file = Path(detection["File"]).name if "/" in str(detection["File"]) else detection["File"]
    gt_file = gt_row["File"]
    
    if det_file != gt_file:
        return False

    # 2. Comparar smell (normalizado)
    detection_smell = normalize_smell_name(detection["Code_Smell"])
    gt_smell = normalize_smell_name(gt_row["Code_Smell"])

    if detection_smell != gt_smell:
        return False

    # 3. Comparar método - MAS ignorar se um deles é 'file_level' ou vazio
    detection_method = str(detection["Method"]).strip() if pd.notna(detection["Method"]) else ""
    gt_method = str(gt_row["Method"]).strip() if pd.notna(gt_row["Method"]) else ""
    
    # Se um dos métodos é 'file_level' ou vazio, não verificar método
    if detection_method.lower() != "file_level" and gt_method.lower() != "file_level":
        if detection_method and gt_method:
            if detection_method != gt_method:
                return False

    # 4. Verificar linha com tolerância
    gt_has_line = pd.notna(gt_row["Line_no"]) or (
        pd.notna(gt_row["Start_line"]) and pd.notna(gt_row["End_line"])
    )
    detection_has_line = pd.notna(detection["Line_no"]) or (
        pd.notna(detection["Start_line"]) and pd.notna(detection["End_line"])
    )

    if gt_has_line and detection_has_line:
        # Caso 1: GT tem range (Start_line/End_line)
        if pd.notna(gt_row["Start_line"]) and pd.notna(gt_row["End_line"]):
            if pd.notna(detection["Line_no"]):
                # Detecção tem Line_no: verificar se está dentro do range do GT
                if not (
                    gt_row["Start_line"] <= detection["Line_no"] <= gt_row["End_line"]
                ):
                    return False
            elif pd.notna(detection["Start_line"]) and pd.notna(detection["End_line"]):
                # Detecção tem range: verificar sobreposição de ranges
                if not (
                    detection["Start_line"] <= gt_row["End_line"]
                    and detection["End_line"] >= gt_row["Start_line"]
                ):
                    return False

        # Caso 2: GT tem apenas Line_no
        elif pd.notna(gt_row["Line_no"]):
            if pd.notna(detection["Line_no"]):
                # Ambos têm Line_no: verificar com tolerância
                if abs(float(gt_row["Line_no"]) - float(detection["Line_no"])) > LINE_TOLERANCE:
                    return False
            elif pd.notna(detection["Start_line"]) and pd.notna(detection["End_line"]):
                # Detecção tem range: verificar se Line_no do GT está dentro do range
                if not (
                    detection["Start_line"]
                    <= gt_row["Line_no"]
                    <= detection["End_line"]
                ):
                    return False

    return True


def find_matches(detections_df, gt_df):
    """
    Encontra matches entre detecções e ground truth.
    
    Cada item do GT só pode ser matched uma vez (evita duplicatas).

    Returns:
        dict com:
            - matches: lista de tuplas (detection_idx, gt_idx)
            - matched_detections: set de índices de detecções que fizeram match
            - matched_gt: set de índices do GT que fizeram match
    """
    matches = []
    matched_detections = set()
    matched_gt = set()

    for det_idx, detection in detections_df.iterrows():
        for gt_idx, gt_row in gt_df.iterrows():
            # Evitar que o mesmo GT seja matched múltiplas vezes
            if gt_idx not in matched_gt and match_detection(detection, gt_row):
                matches.append((det_idx, gt_idx))
                matched_detections.add(det_idx)
                matched_gt.add(gt_idx)
                break

    return {
        "matches": matches,
        "matched_detections": matched_detections,
        "matched_gt": matched_gt,
    }


def calculate_metrics(detections_df, gt_df, matches_result):
    """
    Calcula métricas de validação.

    Returns:
        dict com métricas
    """
    total_detections = len(detections_df)
    total_gt = len(gt_df)

    tp = len(matches_result["matches"])
    fp = total_detections - tp
    fn = total_gt - tp

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1_score = (
        2 * (precision * recall) / (precision + recall)
        if (precision + recall) > 0
        else 0.0
    )

    return {
        "TP": tp,
        "FP": fp,
        "FN": fn,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1_score,
        "Total_Detections": total_detections,
        "Total_GT": total_gt,
    }


print("Funções de validação carregadas!")


Funções de validação carregadas!


## 5. Calcular Métricas


In [130]:
# Encontrar matches entre Complete Prompts e Ground Truth
print("Buscando matches...")
matches_result = find_matches(df_complete_prompts, df_gt)

# Calcular métricas
metrics = calculate_metrics(df_complete_prompts, df_gt, matches_result)

print(f"\nMatches encontrados: {len(matches_result['matches'])}")
print(f"Detecções com match: {len(matches_result['matched_detections'])}")
print(f"Itens do GT com match: {len(matches_result['matched_gt'])}")


Buscando matches...

Matches encontrados: 261
Detecções com match: 261
Itens do GT com match: 261


## 6. Resultados e Análise


In [131]:
# Exibir métricas gerais
print("=" * 80)
print("MÉTRICAS DE VALIDAÇÃO - COMPLETE PROMPTS")
print("=" * 80)
print(f"\nTotal de detecções do sistema: {metrics['Total_Detections']}")
print(f"Total de itens no Ground Truth: {metrics['Total_GT']}")
print(f"\nTrue Positives (TP):  {metrics['TP']}")
print(f"False Positives (FP): {metrics['FP']}")
print(f"False Negatives (FN): {metrics['FN']}")
print(f"\nPrecision: {metrics['Precision']:.4f} ({metrics['Precision'] * 100:.2f}%)")
print(f"Recall:    {metrics['Recall']:.4f} ({metrics['Recall'] * 100:.2f}%)")
print(f"F1-Score:  {metrics['F1-Score']:.4f} ({metrics['F1-Score'] * 100:.2f}%)")

if metrics["Precision"] >= 0.90:
    print("\n✓ Objetivo de 90% de precisão ALCANÇADO!")
else:
    print(
        f"\n⚠ Objetivo de 90% de precisão NÃO alcançado (faltam {0.90 - metrics['Precision']:.4f})"
    )


MÉTRICAS DE VALIDAÇÃO - COMPLETE PROMPTS

Total de detecções do sistema: 432
Total de itens no Ground Truth: 397

True Positives (TP):  261
False Positives (FP): 171
False Negatives (FN): 136

Precision: 0.6042 (60.42%)
Recall:    0.6574 (65.74%)
F1-Score:  0.6297 (62.97%)

⚠ Objetivo de 90% de precisão NÃO alcançado (faltam 0.2958)


In [132]:
# Análise por tipo de smell
print("\n" + "=" * 80)
print("ANÁLISE POR TIPO DE SMELL")
print("=" * 80)

# Agrupar por tipo de smell no GT
gt_by_smell = df_gt.groupby("Code_Smell").size().sort_values(ascending=False)
detections_by_smell = (
    df_complete_prompts.groupby("Code_Smell").size().sort_values(ascending=False)
)

print("\nDistribuição no Ground Truth:")
for smell, count in gt_by_smell.items():
    print(f"  {smell}: {count}")

print("\nDistribuição nas Detecções:")
for smell, count in detections_by_smell.items():
    print(f"  {smell}: {count}")
 
smell_metrics = {}
for smell in gt_by_smell.index:
    smell_normalized = normalize_smell_name(smell)

    gt_filtered = df_gt[
        df_gt["Code_Smell"].apply(normalize_smell_name) == smell_normalized
    ]
    detections_filtered = df_complete_prompts[
        df_complete_prompts["Code_Smell"].apply(normalize_smell_name)
        == smell_normalized
    ]

    if len(detections_filtered) > 0:
        matches_smell = find_matches(detections_filtered, gt_filtered)
        metrics_smell = calculate_metrics(
            detections_filtered, gt_filtered, matches_smell
        )
        smell_metrics[smell] = metrics_smell

print("\n" + "-" * 80)
print("Métricas por Tipo de Smell:")
print("-" * 80)
for smell, metrics_smell in smell_metrics.items():
    print(f"\n{smell}:")
    print(
        f"  TP: {metrics_smell['TP']}, FP: {metrics_smell['FP']}, FN: {metrics_smell['FN']}"
    )
    print(
        f"  Precision: {metrics_smell['Precision']:.4f} ({metrics_smell['Precision'] * 100:.2f}%)"
    )
    print(
        f"  Recall:    {metrics_smell['Recall']:.4f} ({metrics_smell['Recall'] * 100:.2f}%)"
    )
    print(
        f"  F1-Score:  {metrics_smell['F1-Score']:.4f} ({metrics_smell['F1-Score'] * 100:.2f}%)"
    )



ANÁLISE POR TIPO DE SMELL

Distribuição no Ground Truth:
  Long Statement: 79
  Magic Number: 71
  Complex Method: 53
  Long Lambda Function: 43
  Long Method: 40
  Long Message Chain: 38
  Long Identifier: 35
  Empty Catch Block: 19
  Long Parameter List: 12
  Complex Conditional: 6
  Missing Default: 1

Distribuição nas Detecções:
  Magic number: 79
  Long statement: 74
  Long message chain: 53
  Long identifier: 50
  Complex method: 48
  Long lambda function: 42
  Long method: 36
  Long parameter list: 19
  Empty catch block: 17
  Complex conditional: 13
  Missing default: 1

--------------------------------------------------------------------------------
Métricas por Tipo de Smell:
--------------------------------------------------------------------------------

Long Statement:
  TP: 41, FP: 33, FN: 38
  Precision: 0.5541 (55.41%)
  Recall:    0.5190 (51.90%)
  F1-Score:  0.5359 (53.59%)

Magic Number:
  TP: 23, FP: 56, FN: 48
  Precision: 0.2911 (29.11%)
  Recall:    0.3239 (32.3

In [133]:
# Identificar False Positives (detecções que não fizeram match)
fp_indices = set(df_complete_prompts.index) - matches_result["matched_detections"]
fp_df = df_complete_prompts.loc[list(fp_indices)].copy()

print("\n" + "=" * 80)
print(f"FALSE POSITIVES ({len(fp_df)} detecções)")
print("=" * 80)
print("\nPrimeiros 20 False Positives:")
print(
    fp_df[["File", "Method", "Code_Smell", "Line_no", "Start_line", "End_line"]]
    .head(20)
    .to_string()
)

# Contar FPs por tipo de smell
if len(fp_df) > 0:
    print("\nFalse Positives por tipo de smell:")
    fp_by_smell = fp_df.groupby("Code_Smell").size().sort_values(ascending=False)
    for smell, count in fp_by_smell.items():
        print(f"  {smell}: {count}")



FALSE POSITIVES (171 detecções)

Primeiros 20 False Positives:
                           File           Method           Code_Smell  Line_no  Start_line  End_line
6   ff_ippo_store_experience.py   _actor_loss_fn  Long parameter list      163        <NA>      <NA>
7   ff_ippo_store_experience.py  _critic_loss_fn  Long parameter list      193        <NA>      <NA>
8   ff_ippo_store_experience.py   get_learner_fn       Long statement      228        <NA>      <NA>
9   ff_ippo_store_experience.py    learner_setup       Long statement      364        <NA>      <NA>
10  ff_ippo_store_experience.py   run_experiment       Long statement      673        <NA>      <NA>
17  ff_ippo_store_experience.py       file_level      Long identifier      547        <NA>      <NA>
18  ff_ippo_store_experience.py       file_level      Long identifier      574        <NA>      <NA>
21  ff_ippo_store_experience.py  _get_advantages         Magic number      148        <NA>      <NA>
23  ff_ippo_store_experienc

In [134]:
# Identificar False Negatives (itens do GT não detectados)
fn_indices = set(df_gt.index) - matches_result["matched_gt"]
fn_df = df_gt.loc[list(fn_indices)].copy()

print("\n" + "=" * 80)
print(f"FALSE NEGATIVES ({len(fn_df)} itens do GT não detectados)")
print("=" * 80)
print("\nPrimeiros 20 False Negatives:")
print(
    fn_df[["File", "Method", "Code_Smell", "Line_no", "Start_line", "End_line"]]
    .head(20)
    .to_string()
)

# Contar FNs por tipo de smell
if len(fn_df) > 0:
    print("\nFalse Negatives por tipo de smell:")
    fn_by_smell = fn_df.groupby("Code_Smell").size().sort_values(ascending=False)
    for smell, count in fn_by_smell.items():
        print(f"  {smell}: {count}")



FALSE NEGATIVES (136 itens do GT não detectados)

Primeiros 20 False Negatives:
                File                     Method            Code_Smell  Line_no  Start_line  End_line
6      _codespell.py                        NaN       Long Identifier     <NA>        <NA>      <NA>
7      _codespell.py  parse_ignore_words_option       Long Identifier     <NA>        <NA>      <NA>
8      _codespell.py                 parse_file       Long Identifier     <NA>        <NA>      <NA>
9      _codespell.py                       main       Long Identifier     <NA>        <NA>      <NA>
14     _codespell.py           ask_for_word_fix    Long Message Chain      791        <NA>      <NA>
15     _codespell.py           ask_for_word_fix    Long Message Chain      799        <NA>      <NA>
24     _codespell.py                        NaN          Magic Number      142        <NA>      <NA>
25     _codespell.py                        NaN          Magic Number      143        <NA>      <NA>
26     _co